## Este notebook realiza tranformaciones y crea salida para el input de la calculadora de entropia.

### Visitor type 1

1. Filtrar por cuestionario completo: eliminar observaciones donde "ultima pagina" != 7.
2. Filtrar alojamiento: quitar filas con "alojamiento" == 'Ninguno, me desplacé desde mi lugar de residencia'.
3. Clasificar procedencia en 3 categorías:
Internacional: "residencia" == 'Extranjero' o "pais" distinto de "España".
Nacional: "residencia" == 'España' y "provincia" distinta de "Valencia".
Local: "residencia" == 'España' y "provincia" == 'Valencia'.
4. Convertir "noches_valencia":
Valores 1,2,3…9 → enteros.
"mas de 10" → imputar como 10 (para promedios sirve como proxy).
5. Calcular métricas:
% visitor type 1: proporción de cada categoría respecto al total.
average days of attendance: media de "noches_valencia" en cada categoría.
6. Construir dataframe final con columnas:
"visitor type 1"
"% visitor type 1"
"average days of attendance"

In [1]:
# import pandas as pd
import numpy as np
import pandas as pd

In [2]:
# Cargar datos
archivo = "/Users/user/projects/projects/zentropyETL/data/encuesta_procesada.xlsx"

encuesta_procesada = pd.read_excel(archivo)

In [3]:
# Filtrar cuestionarios completos
df = encuesta_procesada.copy()
df = df[df["Última página"] == 7]

In [4]:
# Eliminar quienes usaron su lugar de residencia habitual como alojamiento (no se consideran turistas)
df = df[df["alojamiento"] != "Ninguno, me desplacé desde mi lugar de residencia"]
df["alojamiento"].unique()

array(['Hotel 4 estrellas', 'Hotel 5 estrellas',
       'Apartamento de alquiler (AirBnb)', 'Alojamiento local sin coste',
       'Hotel 3 estrellas', 'Hotel 2 estrellas', 'Pensión o hostal'],
      dtype=object)

In [5]:
print(df["residencia"].unique())
print(df["pais"].unique())
print(df["provincia"].unique())

['Extranjero' 'España']
['Denmark' 'Austria' nan 'Italy' 'France' 'Japan' 'Slovenia'
 'United Kingdom of Great Britain and Northern Ireland' 'Portugal'
 'Mexico' 'Costa Rica']
[nan 'Valencia/València' 'Castellón/Castelló' 'Barcelona' 'Murcia']


In [6]:
# Crear variable "visitor type 1"
"""
- Internacional: residencia = Extranjero \
- Nacional: residencia = España y provincia != Valencia \
- Local: residencia = España y provincia = Valencia
"""
def classify_visitor(row):
    if row["residencia"] == "Extranjero" or (pd.notna(row["pais"]) and row["pais"] != "España"):
        return "International"
    elif row["residencia"] == "España" and row["provincia"] == "Valencia/València":
        return "Local"
    elif row["residencia"] == "España":
        return "National"
    else:
        return np.nan  # por si queda algo ambiguo

df["visitor type 1"] = df.apply(classify_visitor, axis=1)



In [7]:
df["visitor type 1"].unique()

array(['International', 'Local', 'National'], dtype=object)

In [8]:
df["noches_valencia"].unique()

array([2, 3, 7, 'más de 10', 4, 5, 1], dtype=object)

In [9]:
# Convertir noches_valencia a numérico

def parse_noches(val):
    if pd.isna(val):
        return np.nan
    val = str(val).strip().lower()
    if val.isdigit():
        return int(val)
    elif "más de 10" in val:
        return 10
    else:
        return np.nan

df["noches_valencia_num"] = df["noches_valencia"].apply(parse_noches)

In [10]:
# Calcular métricas
# - % visitor type 1 = count / total * 100
# - average days of attendance = mean(noches_valencia_num) por categoría
summary = (
    df.groupby("visitor type 1")
    .agg(
        count_visitors=("visitor type 1", "size"),
        average_days=("noches_valencia_num", "mean")
    )
    .reset_index()
)

# Calcular proporciones
total_visitors = summary["count_visitors"].sum()
summary["% visitor type 1"] = (summary["count_visitors"] / total_visitors).round(2)

# Renombrar columnas finales
summary = summary.rename(columns={
    "visitor type 1": "visitor type 1",
    "average_days": "average days of attendance"
})[["visitor type 1", "% visitor type 1", "average days of attendance"]]

print(summary)

  visitor type 1  % visitor type 1  average days of attendance
0  International              0.81                    4.318182
1          Local              0.07                    5.000000
2       National              0.11                    2.333333


## Desplazamientos diarios a Congreso

1. **Crear variables de uso de transporte hacia el congreso**:

   * `public_transport_use_to_congress = uso_bus_congreso + uso_metro_congreso`
   * `car_use_to_congress = uso_taxi_congreso + uso_coche_congreso`
   * `walk_bike_use_to_congress = uso_bici_congreso + uso_pie_congreso`

2. **Agrupar por `visitor type 1`** y calcular:

   * Promedio de cada variable (`mean`).

3. **Construir un df** con columnas:

   * `visitor type 1`
   * `avg_public_transport_use_to_congress`
   * `avg_car_use_to_congress`
   * `avg_walk_bike_use_to_congress`



In [11]:
# Crear variables de transporte hacia el congreso - combinamos modos de transporte en 3 categorías 
df["public_transport_use_to_congress"] = df["uso_bus_congreso"].fillna(0) + df["uso_metro_congreso"].fillna(0)
df["car_use_to_congress"] = df["uso_taxi_congreso"].fillna(0) + df["uso_coche_congreso"].fillna(0)
df["walk_bike_use_to_congress"] = df["uso_bici_congreso"].fillna(0) + df["uso_pie_congreso"].fillna(0)

In [12]:
# Crear variables de transporte fuera del congreso
df["public_transport_use_ocio"] = df["uso_bus_ocio"].fillna(0) + df["uso_metro_ocio"].fillna(0)
df["car_use_ocio"] = df["uso_taxi_ocio"].fillna(0) + df["uso_coche_ocio"].fillna(0)
df["walk_bike_use_ocio"] = df["uso_bici_ocio"].fillna(0) + df["uso_pie_ocio"].fillna(0)

In [13]:
# Calcular promedios por visitor type 1 
transport_summary = (
    df.groupby("visitor type 1")
    .agg(
        avg_public_transport_use_to_congress=("public_transport_use_to_congress", "mean"),
        avg_car_use_to_congress=("car_use_to_congress", "mean"),
        avg_walk_bike_use_to_congress=("walk_bike_use_to_congress", "mean"),
        avg_public_transport_use_ocio=("public_transport_use_ocio", "mean"),
        avg_car_use_ocio=("car_use_ocio", "mean"),
        avg_walk_bike_ocio=("walk_bike_use_ocio", "mean")
    )
    .reset_index()
)

print(transport_summary)

  visitor type 1  avg_public_transport_use_to_congress  \
0  International                              3.909091   
1          Local                              3.000000   
2       National                              3.000000   

   avg_car_use_to_congress  avg_walk_bike_use_to_congress  \
0                 0.909091                       1.909091   
1                 2.500000                       2.500000   
2                 2.000000                       0.000000   

   avg_public_transport_use_ocio  avg_car_use_ocio  avg_walk_bike_ocio  
0                       2.772727          0.590909            2.772727  
1                       2.000000          1.000000            1.500000  
2                       0.666667          0.000000            2.000000  


In [25]:
# Clasificar por tipo de transporte dominante

def classify_visitor_type2(row):
    # valores de uso de transporte
    public = row["public_transport_use_to_congress"]
    car = row["car_use_to_congress"]
    walk_bike = row["walk_bike_use_to_congress"]

    # si no hay viajes registrados
    if public == 0 and car == 0 and walk_bike == 0:
        return "No transport reported"

    # encontrar transporte dominante
    max_val = max(public, car, walk_bike)

    # si hay empates, aplicamos jerarquía: walk/bike > public > car
    if walk_bike == max_val:
        return "Eco-conscious"
    elif public == max_val:
        return "Young professional/student"
    else:
        return "Standard"

# Aplicar clasificación 
df["visitor type 2"] = df.apply(classify_visitor_type2, axis=1)

# Resumen por visitor type 2
visitor_type2_summary = (
    df["visitor type 2"]
    .value_counts(normalize=True)
).reset_index()

visitor_type2_summary.columns = ["visitor type 2", "% visitor type 2"]

print(visitor_type2_summary)



               visitor type 2  % visitor type 2
0  Young professional/student          0.481481
1               Eco-conscious          0.296296
2                    Standard          0.222222


In [27]:
# Agrupar por visitor type 1 y 2
summary_df = (
    df.groupby(["visitor type 1", "visitor type 2"])
    .agg(
        count_visitors=("visitor type 1", "size"),
        average_days=("noches_valencia_num", "mean"),
        avg_public_transport_use_to_congress=("public_transport_use_to_congress", "mean"),
        avg_car_use_to_congress=("car_use_to_congress", "mean"),
        avg_walk_bike_use_to_congress=("walk_bike_use_to_congress", "mean")
    )
    .reset_index()
)

# Calcular % visitor type 1 (global)
total_visitors = df.shape[0]
type1_percent = (df["visitor type 1"].value_counts(normalize=True)).to_dict()
summary_df["% visitor type 1"] = summary_df["visitor type 1"].map(type1_percent).round(2)

# Calcular % visitor type 2 (dentro de cada visitor type 1)
# Primero total por cada visitor type 1
type2_within_type1 = (
    summary_df.groupby("visitor type 1")["count_visitors"].transform(lambda x: x / x.sum())
)
summary_df["% visitor type 2"] = type2_within_type1.round(2)

# Reordenar columnas finales
summary_df = summary_df[
    [
        "visitor type 1",
        "% visitor type 1",
        "visitor type 2",
        "% visitor type 2",
        "average_days",
        "avg_public_transport_use_to_congress",
        "avg_car_use_to_congress",
        "avg_walk_bike_use_to_congress"
    ]
].rename(columns={"average_days": "average days of attendance"})

print(summary_df)


  visitor type 1  % visitor type 1              visitor type 2  \
0  International              0.81               Eco-conscious   
1  International              0.81                    Standard   
2  International              0.81  Young professional/student   
3          Local              0.07                    Standard   
4          Local              0.07  Young professional/student   
5       National              0.11                    Standard   
6       National              0.11  Young professional/student   

   % visitor type 2  average days of attendance  \
0              0.36                    3.625000   
1              0.14                    3.000000   
2              0.50                    5.181818   
3              0.50                    7.000000   
4              0.50                    3.000000   
5              0.67                    1.500000   
6              0.33                    4.000000   

   avg_public_transport_use_to_congress  avg_car_use_to_congre

In [28]:
# MultiIndex con todas las combinaciones posibles.
import itertools

# definir categorías obligatorias ---
visitor_type1_levels = ["International", "National", "Local"]
visitor_type2_levels = ["Eco-conscious", "Standard", "Young professional/student"]

# Crear todas las combinaciones posibles
all_combinations = pd.MultiIndex.from_product(
    [visitor_type1_levels, visitor_type2_levels],
    names=["visitor type 1", "visitor type 2"]
)

# reindexar summary_df
summary_df = summary_df.set_index(["visitor type 1", "visitor type 2"])
summary_df = summary_df.reindex(all_combinations, fill_value=0).reset_index()

# ajustar % visitor type 1 y 2
# Si los grupos no existían, poner 0 en porcentajes también
summary_df["% visitor type 1"] = summary_df["% visitor type 1"].fillna(0)
summary_df["% visitor type 2"] = summary_df["% visitor type 2"].fillna(0)

# llenar con 0 promedios de noches/transporte si no hay datos ---
numeric_cols = [
    "average days of attendance",
    "avg_public_transport_use_to_congress",
    "avg_car_use_to_congress",
    "avg_walk_bike_use_to_congress"
] + [col for col in summary_df.columns if col.startswith("avg_nights_")]

summary_df[numeric_cols] = summary_df[numeric_cols].fillna(0)

print(summary_df)

  visitor type 1              visitor type 2  % visitor type 1  \
0  International               Eco-conscious              0.81   
1  International                    Standard              0.81   
2  International  Young professional/student              0.81   
3       National               Eco-conscious              0.00   
4       National                    Standard              0.11   
5       National  Young professional/student              0.11   
6          Local               Eco-conscious              0.00   
7          Local                    Standard              0.07   
8          Local  Young professional/student              0.07   

   % visitor type 2  average days of attendance  \
0              0.36                    3.625000   
1              0.14                    3.000000   
2              0.50                    5.181818   
3              0.00                    0.000000   
4              0.67                    1.500000   
5              0.33              

In [29]:
# Agrupar por visitor type 1, visitor type 2 y alojamiento.
# Calcular la media de noches_valencia_num.
avg_nights_by_accommodation = (
    df.groupby(["visitor type 1", "visitor type 2", "alojamiento"])["noches_valencia_num"]
    .mean()
    .reset_index()
)

# Pivotear para tener una fila por cada combinación de visitor type 1, visitor type 2 y alojamiento.
avg_nights_pivot = avg_nights_by_accommodation.pivot(
    index=["visitor type 1", "visitor type 2"], 
    columns="alojamiento",
    values="noches_valencia_num"
).reset_index()

# Renombrar columnas
avg_nights_pivot = avg_nights_pivot.rename(
    columns=lambda x: f"avg_nights_{x}" if x not in ["visitor type 1", "visitor type 2"] else x
    )

summary_df = pd.merge(
    summary_df,
    avg_nights_pivot,
    on=["visitor type 1", "visitor type 2"],
    how="left"
)

print(summary_df)

  visitor type 1              visitor type 2  % visitor type 1  \
0  International               Eco-conscious              0.81   
1  International                    Standard              0.81   
2  International  Young professional/student              0.81   
3       National               Eco-conscious              0.00   
4       National                    Standard              0.11   
5       National  Young professional/student              0.11   
6          Local               Eco-conscious              0.00   
7          Local                    Standard              0.07   
8          Local  Young professional/student              0.07   

   % visitor type 2  average days of attendance  \
0              0.36                    3.625000   
1              0.14                    3.000000   
2              0.50                    5.181818   
3              0.00                    0.000000   
4              0.67                    1.500000   
5              0.33              

In [30]:
# Agregar por tipo de comida 
food_cols = {
    "carne_roja": ["carne_roja_restaurante", "carne_roja_fastfood", "carne_roja_domicilio", "carne_roja_casera"],
    "avepescado": ["avepescado_restaurante", "avepescado_fastfood", "avepescado_domicilio", "avepescado_casera"],
    "marisco": ["marisco_restaurante", "marisco_domicilio", "marisco_casera"]
}

# Crear totales por tipo de comida ---
df["total_carne_roja"] = df[food_cols["carne_roja"]].sum(axis=1)
df["total_avepescado"] = df[food_cols["avepescado"]].sum(axis=1)
df["total_marisco"] = df[food_cols["marisco"]].sum(axis=1)

In [31]:
# Crear totales por lugar de comida
df["total_restaurante"] = df[["carne_roja_restaurante", "avepescado_restaurante", "marisco_restaurante"]].sum(axis=1)
df["total_fastfood"]   = df[["carne_roja_fastfood", "avepescado_fastfood"]].sum(axis=1)
df["total_domicilio"]  = df[["carne_roja_domicilio", "avepescado_domicilio", "marisco_domicilio"]].sum(axis=1)
df["total_casera"]     = df[["carne_roja_casera", "avepescado_casera", "marisco_casera"]].sum(axis=1)


In [32]:
# agrupar por visitor type 1 y visitor type 2 para sacar promedios ---
avg_food_totals = (
    df.groupby(["visitor type 1", "visitor type 2"])[
        ["total_carne_roja", "total_avepescado", "total_marisco",
         "total_restaurante", "total_fastfood", "total_domicilio", "total_casera"]
    ]
    .mean()
    .reset_index()
)

print(avg_food_totals)

# merge con summary_df ---
summary_df = pd.merge(summary_df, avg_food_totals, on=["visitor type 1", "visitor type 2"], how="left")


  visitor type 1              visitor type 2  total_carne_roja  \
0  International               Eco-conscious          3.125000   
1  International                    Standard          2.333333   
2  International  Young professional/student          2.000000   
3          Local                    Standard          1.000000   
4          Local  Young professional/student          4.000000   
5       National                    Standard          1.500000   
6       National  Young professional/student          0.000000   

   total_avepescado  total_marisco  total_restaurante  total_fastfood  \
0          2.250000       1.625000           5.125000        0.750000   
1          1.333333       1.000000           3.666667        0.333333   
2          1.454545       1.818182           3.818182        0.272727   
3          2.000000       0.000000           3.000000        0.000000   
4          6.000000       7.000000           4.000000        0.000000   
5          0.000000       0.50000

In [33]:
# Compras
# cambiar cantidad compras a numerico
# Columnas a convertir
cols_to_convert = ["compras_textiles_cantidad",
    "compras_artesania_cantidad",
    "compras_alimentacion_cantidad",
    "compras_souvenirs_cantidad"]

replace_dict = {'más de 5': 5}

for col in cols_to_convert:
    # Primero reemplazamos 'más de 5'
    df[col] = df[col].replace(replace_dict)
    # Convertimos a numérico y rellenamos NA con 0
    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)


In [34]:
# Agrupar por visitor_type y calcular promedio
avg_compras_cantiddad = df.groupby(['visitor type 1', 'visitor type 2'])[cols_to_convert].mean().reset_index()

summary_df = pd.concat([summary_df, avg_compras_cantiddad], axis=1)

print(summary_df)

   visitor type 1              visitor type 2  % visitor type 1  \
0   International               Eco-conscious              0.81   
1   International                    Standard              0.81   
2   International  Young professional/student              0.81   
3        National               Eco-conscious              0.00   
4        National                    Standard              0.11   
5        National  Young professional/student              0.11   
6           Local               Eco-conscious              0.00   
7           Local                    Standard              0.07   
8           Local  Young professional/student              0.07   

   % visitor type 2  average days of attendance  \
0              0.36                    3.625000   
1              0.14                    3.000000   
2              0.50                    5.181818   
3              0.00                    0.000000   
4              0.67                    1.500000   
5              0.33    

In [35]:
# Exportar a Excel con pestañas separadas
output_path = "/Users/user/projects/projects/zentropyETL/data/visitor_summary.xlsx"

with pd.ExcelWriter(output_path, engine="openpyxl") as writer:
    # Sheet 1 con resumen de métricas agregadas
    summary_df.to_excel(writer, sheet_name="visitor_summary", index=False)
    
    # Sheet 2 con dataframe completo transformado
    df.to_excel(writer, sheet_name="full_transformed_data", index=False)

print(f"Archivo Excel exportado correctamente a: {output_path}")


Archivo Excel exportado correctamente a: /Users/user/projects/projects/zentropyETL/data/visitor_summary.xlsx
